In [ ]:
"""
Created on Mon Oct 21 16:13 2024

Prepare the Justine file containing zmin and zmax

@author: Clara Burgard
"""

In [1]:
import xarray as xr
from tqdm.notebook import tqdm

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

In [4]:
file_Justine = xr.open_dataset(inputpath_raw + 'Mask_Iceshelf_IMBIE2_v2.nc')
BedMachine_mask_2km= xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_masks_only.nc')
BedMachine_vars_2km= xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_allvars.nc')


In [5]:
BedMachine_mask_2km

<xarray.Dataset>
Dimensions:       (x: 3041, y: 3041)
Coordinates:
  * x             (x) int32 -3040000 -3038000 -3036000 ... 3038000 3040000
  * y             (y) int32 -3040000 -3038000 -3036000 ... 3038000 3040000
Data variables:
    mask_0_1_2    (y, x) float64 ...
    ocean_conc    (y, x) float64 ...
    ground_conc   (y, x) float64 ...
    isf_conc      (y, x) float64 ...
    icefree_conc  (y, x) float64 ...

In [12]:
ice_draft = (BedMachine_vars_2km['thickness'] - BedMachine_vars_2km['surface'])
ice_draft = ice_draft.where(ice_draft >= 0)

In [25]:
ice_draft.plot()

In [9]:
file_Justine['Iceshelf'].plot(vmax=10)

In [14]:
kisf = 8
ice_draft.where(file_Justine['Iceshelf'] == kisf).max()

<xarray.DataArray ()>
array(918.57999754)

In [29]:
zmin_list = []
zmax_list = []
perc_01_list = []
perc_99_list = []
for kisf in tqdm(file_Justine['ID']):
    zmin = ice_draft.where(file_Justine['Iceshelf'] == kisf).min()
    zmin_list.append(zmin.assign_coords({'ID': kisf}))
    zmax = ice_draft.where(file_Justine['Iceshelf'] == kisf).max()
    zmax_list.append(zmax.assign_coords({'ID': kisf}))
    perc_01 = ice_draft.where(file_Justine['Iceshelf'] == kisf).quantile(0.01)
    perc_01_list.append(perc_01.assign_coords({'ID': kisf}))
    perc_99 = ice_draft.where(file_Justine['Iceshelf'] == kisf).quantile(0.99)
    perc_99_list.append(perc_99.assign_coords({'ID': kisf}))

zmin_all = xr.concat(zmin_list, dim='ID')
zmax_all = xr.concat(zmax_list, dim='ID')
perc_01_all = xr.concat(perc_01_list, dim='ID')
perc_99_all = xr.concat(perc_99_list, dim='ID')

  0%|          | 0/133 [00:00<?, ?it/s]

/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_bas

In [37]:
file_Justine['z_min'] = zmin_all
file_Justine['z_min'].attrs['standard_name'] = 'most shallow point of the ice-shelf draft'
file_Justine['z_min'].attrs['units'] = 'm below sea level'

file_Justine['z_max'] = zmax_all
file_Justine['z_max'].attrs['standard_name'] = 'deepest point of the ice-shelf draft'
file_Justine['z_max'].attrs['units'] = 'm below sea level'

file_Justine['z_perc01'] = perc_01_all
file_Justine['z_perc01'].attrs['standard_name'] = '1st percentile over ice-shelf draft'
file_Justine['z_perc01'].attrs['units'] = 'm below sea level'

file_Justine['z_perc99'] = perc_99_all
file_Justine['z_perc99'].attrs['standard_name'] = '99th percentile over ice-shelf draft'
file_Justine['z_perc99'].attrs['units'] = 'm below sea level'

file_Justine['isf_conc'] = BedMachine_mask_2km['isf_conc']
file_Justine['isf_conc'].attrs['standard_name'] = 'Ice-shelf concentration in the grid cell between 0 and 1'

In [39]:
file_Justine.to_netcdf(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc')